<a href="https://colab.research.google.com/github/julianorinaldi/CAIA003-DAT003-DataMining/blob/main/Atividade1/Exercicio1-2/Exerc%C3%ADcio_1%5BTwitter%5D%5BJuliano_Rinaldi_CAIA003_DAT003%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   i- Instale todas as bibliotecas mencionadas na apresentação teórica sobre coleta de dados.
*   ii- Crie uma aplicação no Twitter. Em seguida, gere os credenciais necessárias para acessar dados da
API do Twitter.

1.   Faça uma adaptação no coletor de tweets para coletar qualquer tipo de tweet compartilhado no Brasil. 
Dica: veja a documentação da biblioteca TwitterApi. Faça uma coleta por alguns minutos, salvando o resultado em um arquivo ou em um banco de dados.

2.   Com base no dataset gerado na questão 1, faça:
*   a) Separe todas as hashtags usadas nos Tweets.
*   b) Descubra a porcentagem de tweets com hashtags.
*   c) Identifique a hashtag mais popular.


# Instalação dos Pacotes Twitter

In [65]:
!pip install TwitterAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Importando Bibliotecas

In [66]:
from TwitterAPI import TwitterAPI, TwitterOAuth, TwitterRequestError, TwitterConnectionError, HydrateType, OAuthType
import json

# Desenvolvimento

In [ ]:
file_name="./credentials.txt"
f= open(file_name,"w")
f.write("consumer_key=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
f.write("\n")
f.write("consumer_secret=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
f.write("\n")
f.write("access_token_key=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
f.write("\n")
f.write("access_token_secret=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
f.close()

!cat credentials.txt

1- Faça uma adaptação no coletor de tweets para coletar qualquer tipo de tweet compartilhado no Brasil. 

Dica: veja a documentação da biblioteca TwitterApi. Faça uma coleta por alguns minutos, salvando o resultado em um arquivo ou em um banco de dados.

In [68]:
print('1- Faça uma adaptação no coletor de tweets para coletar qualquer tipo de tweet compartilhado no Brasil. ')
print('Dica: veja a documentação da biblioteca TwitterApi. Faça uma coleta por alguns minutos, salvando o resultado em um arquivo ou em um banco de dados.')

# Filtros
QUERY = 'Brasil'
EXPANSIONS = 'author_id'
TWEET_FIELDS='author_id,text,source'
USER_FIELDS='name,username,entities'

#Quantidade de Twitters deseja coletar
AMOUNT_TWITTERS = 10000

o = TwitterOAuth.read_file(file_name=file_name)
api = TwitterAPI(o.consumer_key, o.consumer_secret, auth_type=OAuthType.OAUTH2, api_version='2')

# Responsável pela limpeza de todas as regras de filtros
r = api.request('tweets/search/stream/rules', method_override='GET')
if r.status_code != 200:
	exit()

rules=r.json()
if not (rules.get('data') is None):
	ids = list(map(lambda rule: rule["id"], rules["data"]))
	payload = {"delete": {"ids": ids}}
	r = api.request('tweets/search/stream/rules', payload)
	print(f'[{r.status_code}] REGRAS DELETADAS: {json.dumps(r.json(), indent=2)}\n')
	if r.status_code != 200:
		exit()

# Adicionar regras ao Stream

r = api.request('tweets/search/stream/rules', {'add': [{'value':QUERY}]})
print(f'[{r.status_code}] REGRAS ADICIONADAS: {json.dumps(r.json(), indent=2)}\n')
if r.status_code != 201:
	exit()

# Lista regras do Stream

r = api.request('tweets/search/stream/rules', method_override='GET')
print(f'[{r.status_code}] REGRAS: {json.dumps(r.json(), indent=2)}\n')
if r.status_code != 200:
	exit()

# Inicia abertura do Stream para pegar Twitters

r = api.request('tweets/search/stream', {
		'expansions': EXPANSIONS,
		'tweet.fields': TWEET_FIELDS,
		'user.fields': USER_FIELDS
	},
	hydrate_type=HydrateType.APPEND)

print(f'[{r.status_code}] INICIANDO STREAM...')
if r.status_code != 200: 
	exit()

count = 0
with open("./tweets_store.json", 'w') as json_file:
	json_file.write("[")
	for item in r:
		count += 1
		#print(json.dumps(item, indent=4, sort_keys=True))
		#print(f'################# Coletado  [{count}] Twitters #########################')
		json.dump(item, json_file, indent=4, separators=(',',': '))
		if count >= AMOUNT_TWITTERS:
			break
		json_file.write(",")
	json_file.write("]")

print(f'################# Total de Twitters coletados: [{count}]#########################')


1- Faça uma adaptação no coletor de tweets para coletar qualquer tipo de tweet compartilhado no Brasil. 
Dica: veja a documentação da biblioteca TwitterApi. Faça uma coleta por alguns minutos, salvando o resultado em um arquivo ou em um banco de dados.
[200] REGRAS DELETADAS: {
  "meta": {
    "sent": "2022-06-13T23:14:16.062Z",
    "summary": {
      "deleted": 1,
      "not_deleted": 0
    }
  }
}

[201] REGRAS ADICIONADAS: {
  "data": [
    {
      "value": "Brasil",
      "id": "1536487143626915841"
    }
  ],
  "meta": {
    "sent": "2022-06-13T23:14:17.531Z",
    "summary": {
      "created": 1,
      "not_created": 0,
      "valid": 1,
      "invalid": 0
    }
  }
}

[200] REGRAS: {
  "data": [
    {
      "id": "1536487143626915841",
      "value": "Brasil"
    }
  ],
  "meta": {
    "sent": "2022-06-13T23:14:17.722Z",
    "result_count": 1
  }
}

[200] INICIANDO STREAM...
################# Total de Twitters coletados: [10000]#########################


2- Com base no dataset gerado na questão 1, faça:

a) Separe todas as hashtags usadas nos Tweets.

b) Descubra a porcentagem de tweets com hashtags.

c) Identifique a hashtag mais popular.

In [69]:
# Função recursiva que busca qualquer nó no Json
def json_extract(obj, key):
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

f = open('tweets_store.json')
data = json.load(f)
f.close()

# Extraindo apenas o que importa, id (identificado do twitter), tag (referente a itens do hashtag)
print('a) Separe todas as hashtags usadas nos Tweets.')
hashtagsDict = {}
for dataItem in data:
  id = dataItem['data']['id']
  tags = json_extract(dataItem, 'tag')
  hashtagsDict[id] = tags

onlyTagsTwittersDict = dict(filter(lambda elem: len(elem[1]) > 0, hashtagsDict.items()))
onlyTagsList = list(onlyTagsTwittersDict.values())
print(f'Hashtags usadas nos Tweets: {onlyTagsList}')

print()

print('b) Descubra a porcentagem de tweets com hashtags.')
# Quantidade de Twitters coletados
allTwitters = len(hashtagsDict)
# Quantidade de Twitters que possuem elementos (tags)
percentTwitterHashtag = len(onlyTagsTwittersDict)/allTwitters * 100
print(f'Porcentagem de tweets com hashtags: {percentTwitterHashtag}%')

print()

print('c) Identifique a hashtag mais popular.')
listTags = []
for tagArray in onlyTagsList:
  for tag in tagArray:
    listTags.append(tag)

listCounter = {i:listTags.count(i) for i in listTags}
listCounterOrdened = sorted(listCounter.items(), key=lambda x: x[1], reverse=True)
print(f'A tag mais popula é: "{listCounterOrdened[0][0]}" com "{listCounterOrdened[0][1]}" repetições')

print()

print('Ordem das Tags mais populares')
print(listCounterOrdened)

a) Separe todas as hashtags usadas nos Tweets.
Hashtags usadas nos Tweets: [['ForaBozo'], ['humanidades', 'comércio', 'futuros', 'artes', 'logística', 'moda', 'varejo', 'tecnologias', 'marketing', 'eClothing', 'wearables', 'história'], ['Ditaduranuncamais', 'ForaBolsonaro', 'ApoioPT', 'lulapresidente2022', 'CIÊNCIA', 'VacinaTodos'], ['MAGA', 'KAG'], ['blacklivesmatter'], ['amoBsb', 'rock', 'SalvemoCerrado', 'Quimicaévida', 'ciênciasalvavidas', 'Army', 'Gamer'], ['bullsnation'], ['Bolsonaro2022'], ['esquerdista', 'antifacista', 'forabolsonaro', 'lula2022'], ['Lula2022', 'VoltaLula'], ['BTS'], ['Lula13'], ['humanidades', 'comércio', 'futuros', 'artes', 'logística', 'moda', 'varejo', 'tecnologias', 'marketing', 'eClothing', 'wearables', 'história'], ['CAMBIACOLOMBIA'], ['EleNão'], ['DubNation'], ['MobileLegends', 'mlbb'], ['Ma10rDoBrasil', 'VerdeÉaCorDaInveja'], ['ActuallyAutistic'], ['Conhecimento', 'ignorância'], ['Mequetrefismos'], ['atleticanosegueatleticano'], ['ForaBolsonaro'], ['Fo